## Contents
1. Importing libraries
2. Reading data, setting up
3. Data wrangling and cleaning
4. Consistency check & cleansing
5. Combining datasets

### 1. Importing libraries

In [18]:
# importing libraries
import pandas as pd
import numpy as np
import os

### 2. Setup

In [19]:
# Task 3.
# setting up file paths, reading data
path_original = r'C:\Users\Richárd\Desktop\CareerFoundry\Data Immersion\Archievment 4\Instacart_Basket_Analysis_2023_04_03\02_Data\Original_Data'
path_prepared = r'C:\Users\Richárd\Desktop\CareerFoundry\Data Immersion\Archievment 4\Instacart_Basket_Analysis_2023_04_03\02_Data\Prepared_Data'
df_customers = pd.read_csv(os.path.join(path_original, 'customers.csv'))
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


### 3. Data wrangling and cleaning

In [20]:
# Task 4. Wrangling
# consistently rename columns
df_customers = df_customers.rename(columns = {'First Name': 'first_name', 'Surnam': 'surname', 'Gender': 'gender', 'STATE': 'state', 'Age': 'age', 'n_dependants': 'number_of_dependants', 'fam_status': 'family_status'})
# There seems to be no reason to include actual name s of the customers - moreover it raises a privacy issue
# creating a subset without first_name and surname columns
df_cust = df_customers.drop(columns = ['first_name', 'surname'])
# checking descriptive statistics
df_cust.head()

,user_id,gender,state,age,date_joined,number_of_dependants,family_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


### 4. Consistency checks and cleansing

In [21]:
# Task 5. Consistency checks
# checking decriptive statistics
df_cust.describe() # everything seems reasonable here

,user_id,age,number_of_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [22]:
# checking datatype consistency
for col in df_cust:
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)
# no mixed type columns found

In [23]:
# checking for duplicates
dups = df_cust.duplicated()
dups.value_counts() # there seem to be no duplicates

False    206209
dtype: int64

In [24]:
# checking for any null values
df_cust.isnull().sum() # no null values found

user_id                 0
gender                  0
state                   0
age                     0
date_joined             0
number_of_dependants    0
family_status           0
income                  0
dtype: int64

###  5. Combining datasets

In [26]:
# Task 6. Combining customer data with the main dataset
# reading data
main_data = pd.read_pickle(os.path.join(path_prepared, 'merged_data_with_groupings.pkl'))
main_data.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_after_previous_order,product_id,add_to_cart_order,reordered,product_name,...,price_range,busiest_day,busiest_days,busiest_period_of_day,max_orders,loyality_flag,avg_spending,spending_flag,median_days_between_orders,order_frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Least busy day,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Least busy day,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [32]:
# joining the datasets
merged_data = main_data.merge(df_cust, on = 'user_id', how = 'inner')
merged_data.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_after_previous_order,product_id,add_to_cart_order,reordered,product_name,...,spending_flag,median_days_between_orders,order_frequency_flag,gender,state,age,date_joined,number_of_dependants,family_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


In [33]:
# Exporting combined dataset to a pickle
merged_data.to_pickle(os.path.join(path_prepared, 'full_data.pkl'))